In [ ]:
# PGC — Detalhe de Catálogo (rápido): paginação otimizada, preview DF, JSON + XLSX
# Endpoint: GET https://dadosabertos.compras.gov.br/modulo-pgc/2_consultarPgcDetalheCatalogo

import os
import re
import json
import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from IPython.display import display  # <- para exibir DF no Jupyter

# ========= CONFIG =========
PASTA_DESTINO = r"C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\5 - PGC"

URL = "https://dadosabertos.compras.gov.br/modulo-pgc/2_consultarPgcDetalheCatalogo"
TOKEN = "1"                                   # <--- substitua
ANO = 2025                                                 # int
TIPO = "Material"                                          # "Material" ou "Servico"
CODIGOS = [8955]                                           # um ou vários ints
TAMANHO_PAGINA = 500                                       # se 400/1000 funcionarem, melhor ainda
MAX_WORKERS = min(8, max(1, len(CODIGOS)))                 # threads por código
PREVIEW_ROWS = 20                                          # quantas linhas mostrar no preview (ajuste à vontade)

# Exibição completa de colunas no preview
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/json",
    "Connection": "keep-alive",
    "Accept-Encoding": "gzip, deflate",
    "User-Agent": "pgc-client/1.0",
}

# ========= Infra de rede =========
def make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update(HEADERS)
    retry = Retry(
        total=5,
        backoff_factor=0.5,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=20, pool_maxsize=20)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

print_lock = threading.Lock()
def log(msg: str):
    with print_lock:
        print(msg, flush=True)

# ========= Fetch por código (com progresso) =========
def fetch_codigo(codigo: int):
    """
    Busca todas as páginas para um 'codigo', imprimindo progresso:
    - Requisição da página
    - Quantidade de itens por página e acumulado
    - Detecção da última página
    Retorna a lista de registros (dicts).
    """
    sess = make_session()
    pagina = 1
    coletados = []
    page_size = TAMANHO_PAGINA  # cópia local (sem global)

    while True:
        params = {
            "pagina": pagina,
            "tamanhoPagina": page_size,
            "anoPcaProjetoCompra": ANO,
            "tipo": TIPO,
            "codigo": int(codigo),
        }

        log(f"↪️  COD {codigo}: requisitando página {pagina} (tamanhoPagina={page_size})...")
        t0 = time.perf_counter()
        r = sess.get(URL, params=params, timeout=(7, 90))
        dt = time.perf_counter() - t0

        # Se tamanhoPagina for alto demais, reduz localmente e tenta a MESMA página de novo
        if r.status_code == 400 and "tamanhoPagina" in r.text.lower():
            new_size = max(100, page_size // 2)
            if new_size == page_size:
                new_size = 100
            log(f"⚠️  COD {codigo}: reduzindo tamanhoPagina {page_size} → {new_size} (HTTP 400).")
            page_size = new_size
            continue

        if r.status_code not in (200, 304):
            log(f"❌ COD {codigo} P{pagina}: HTTP {r.status_code} | {r.text[:180]}")
            break

        try:
            payload = r.json()
        except Exception as e:
            log(f"❌ COD {codigo} P{pagina}: JSON inválido ({e})")
            break

        # Normaliza para lista
        resultados = None
        if isinstance(payload, list):
            resultados = payload
        elif isinstance(payload, dict):
            for k in ("resultado", "resultados", "content", "itens", "items", "dados", "data"):
                v = payload.get(k)
                if isinstance(v, list):
                    resultados = v
                    break
            if resultados is None:
                for v in payload.values():
                    if isinstance(v, list):
                        resultados = v
                        break

        if not resultados:
            log(f"✅ COD {codigo}: fim (P{pagina-1} foi a última).")
            break

        coletados.extend(resultados)
        log(f"🟢 COD {codigo}: página {pagina} extraída ({len(resultados)} itens, {dt:.2f}s) "
            f"— acumulado {len(coletados)}")

        # Última página quando vier menos que o page_size atual
        if len(resultados) < page_size:
            log(f"ℹ️  COD {codigo}: última página detectada (len<{page_size}).")
            break

        pagina += 1

    # metadados úteis
    for b in coletados:
        if isinstance(b, dict):
            b.setdefault("_ano", ANO)
            b.setdefault("_tipo", TIPO)
            b.setdefault("_codigo", codigo)
    return coletados

# ========= Execução paralela por código =========
t_start = time.perf_counter()
todos = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = {ex.submit(fetch_codigo, cod): cod for cod in CODIGOS}
    for fut in as_completed(futures):
        cod = futures[fut]
        try:
            data = fut.result()
            log(f"✅ COD {cod}: concluído com {len(data)} linhas.")
            todos.extend(data)
        except Exception as e:
            log(f"❌ COD {cod}: falhou — {e}")

elapsed = time.perf_counter() - t_start
log(f"\n📦 Total consolidado: {len(todos)} linhas em {elapsed:.1f}s\n")

# ========= Saídas =========
base = f"pgc_detalhe_catalogo_{TIPO}_{ANO}"
if len(CODIGOS) == 1:
    base += f"_cod{CODIGOS[0]}"
json_path = os.path.join(PASTA_DESTINO, f"{base}.json")
xlsx_path = os.path.join(PASTA_DESTINO, f"{base}.xlsx")

# JSON
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(todos, f, ensure_ascii=False, indent=2)
log(f"💾 JSON salvo: {json_path}  ({len(todos)} linhas)")

# XLSX + PREVIEW DO DATAFRAME
if todos:
    df = pd.DataFrame(todos)

    # Preserva possíveis IDs longos como texto
    possiveis_ids = [
        "cnpjFornecedor", "orgaoEntidadeCnpj", "codigoPNCP", "uasg",
        "numeroProcesso", "numeroAviso", "idCompra", "idItemCompra"
    ]
    for col in possiveis_ids:
        if col in df.columns:
            df[col] = df[col].astype(str)

    # Limpeza de caracteres ilegais do Excel (apenas em colunas de texto)
    illegal = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")
    str_cols = df.select_dtypes(include=["object"]).columns
    for c in str_cols:
        try:
            df[c] = df[c].astype("string").str.replace(illegal, "", regex=True)
        except Exception:
            pass

    # ===== PREVIEW DO DF ANTES DE EXPORTAR =====
    print(f"👀 Pré-visualização — linhas: {len(df)} | colunas: {df.shape[1]}")
    print("\nTipos de dados por coluna:")
    print(df.dtypes.sort_index())
    print("\nPrimeiras linhas:")
    display(df.head(PREVIEW_ROWS))   # mostre mais/menos linhas ajustando PREVIEW_ROWS
    

   

↪️  COD 8955: requisitando página 1 (tamanhoPagina=500)...
🟢 COD 8955: página 1 extraída (500 itens, 8.81s) — acumulado 500
↪️  COD 8955: requisitando página 2 (tamanhoPagina=500)...
🟢 COD 8955: página 2 extraída (500 itens, 16.28s) — acumulado 1000
↪️  COD 8955: requisitando página 3 (tamanhoPagina=500)...
🟢 COD 8955: página 3 extraída (500 itens, 25.87s) — acumulado 1500
↪️  COD 8955: requisitando página 4 (tamanhoPagina=500)...
🟢 COD 8955: página 4 extraída (500 itens, 33.08s) — acumulado 2000
↪️  COD 8955: requisitando página 5 (tamanhoPagina=500)...
🟢 COD 8955: página 5 extraída (500 itens, 40.07s) — acumulado 2500
↪️  COD 8955: requisitando página 6 (tamanhoPagina=500)...
🟢 COD 8955: página 6 extraída (1 itens, 41.41s) — acumulado 2501
ℹ️  COD 8955: última página detectada (len<500).
✅ COD 8955: concluído com 2501 linhas.

📦 Total consolidado: 2501 linhas em 165.7s

💾 JSON salvo: C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\5 - PGC\pgc_detalhe_catalogo_Materi

,codigoUasg,nomeUasg,orgao,numeroArtefato,anoArtefato,codigoEstadoArtefato,codigoCategoriaArtefato,descricaoArtefato,codigoTipoArtefato,ordemDfd,descricaoObjetoDfd,nivelPrioridadeDfd,dataPrevistaFormalizacaoDemanda,codigoAreaDfd,tipoItem,itemSustentavel,codigoGrupoMaterial,nomeGrupoMaterial,codigoClasseMaterial,nomeClasseMaterial,codigoPdmMaterial,nomePdmMaterial,codigoSecaoServico,nomeSecaoServico,codigoDivisaoServico,nomeDivisaoServico,codigoGrupoServico,nomeGrupoServico,codigoClasseServico,nomeClasseServico,codigoSubclasseServico,nomeSubclasseServico,codigoItemCatalogo,descricaoItemCatalogo,siglaUnidadeFornecimento,nomeUnidadeFornecimento,quantidadeItem,valorUnitarioItem,valorTotalItem,tituloProjetoCompra,descricaoProjetoCompra,anoPcaProjetoCompra,dataInicioProcessoCompra,dataFimProcessoCompra,duracaoProcessoCompra,numeroItemPncp,statusContratacaoExecucao,dataHoraPublicacaoPncp,dataHoraAtualizacaoArtefato,dataHoraAtualizacaoProjetoCompra,dataHoraAtualizacaoDfd,dataHoraAtualizacaoItem,_ano,_tipo,_codigo
0,928257,CONSELHO DE ARQUITETURA E URBANISMO - PR,14804099000199,1,2025,9,1,1/2025,5,1,Aquisição de matérias de consumo diversos e ut...,1.0,2025-07-30T00:00:00,44523,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,618302,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , TIPO: ÚNIC...",CX,Caixa,300.0,35.0000,10500.000,Aquisição material alimenticio - Ca´fé,"Aquisição de café moído torrado gourmet, embal...",2025,2025-02-03T00:00:00,2025-03-31T00:00:00,56,1,NaN,2025-01-28T10:07:10.702327,2025-01-28T10:07:10.771,2025-06-06T15:24:38,2025-01-28T10:07:09,2025-01-28T10:07:08.97774,2025,Material,8955
1,156678,UNIVERSIDADE FEDERAL DE JATAI,35840659000130,6,2025,9,1,6/2025,5,2,SeTI-Materiais de Copa,1.0,2025-07-31T00:00:00,14348,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463571,"CAFÉ, APRESENTAÇÃO: TORRADO EM GRÃO , INTENSID...",PCT,Pacote,25.0,50.0000,1250.000,Material de Expediente Seti,"Louças, materiais de higiene",2025,2025-02-20T00:00:00,2025-05-28T00:00:00,97,1,NaN,2024-06-13T15:53:56.061181,2024-06-13T15:53:56.145,2024-11-27T15:43:12,2025-05-06T17:32:31,2024-06-13T15:53:54.230632,2025,Material,8955
2,929966,CAMARA MUNICIPAL DE CORDEIRO - RJ,42498600000171,35,2025,9,1,35/2025,5,1,"Aquisição de Material de Copa, Cozinha, Gênero...",0.0,2025-05-30T00:00:00,46426,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463591,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , INTENSIDAD...",PCT,Pacote,60.0,21.0000,1260.000,Aquisição de Material de Copa e Cozinha,<NA>,2025,2025-03-20T00:00:00,2025-06-04T00:00:00,76,1,NaN,2024-05-16T13:45:30.090226,2024-05-16T13:45:30.601,2025-01-10T14:55:51,2024-05-16T13:45:29,2024-05-16T13:45:29.154059,2025,Material,8955
3,380197,ESP-PENITENCIARIA I DE SERRA AZUL,96291141000180,6,2025,9,1,6/2025,5,2,AQUISIÇÃO DE GÊNEROS ALIMENTÍCIOS ESTOCÁVEIS P...,2.0,2025-04-30T00:00:00,39834,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19765,ACHOCOLATADO,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463556,"ACHOCOLATADO, APRESENTAÇÃO: PÓ , SABOR: TRADIC...",EMB,Embalagem,72.0,15.4680,1113.696,GENEROS ALIMENTÍCIOS - JAN-ABR-25,<NA>,2025,2024-10-25T00:00:00,2025-01-02T00:00:00,69,2,NaN,2024-06-11T15:28:13.12969,2024-06-11T15:28:13.566,2024-06-18T16:11:36,2024-12-12T10:52:05,2024-12-12T10:51:46.323878,2025,Material,8955
4,193001,DNOCS 1A.DR - TERESINA-PI,00043711000143,18,2025,9,1,18/2025,5,1,Aquisição de café e açúcar para a CEST/PI.,0.0,2025-12-31T00:00:00,7704,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463587,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , INTENSIDAD...",PCT,Pacote,200.0,11.0000,2200.000,Aquisição de café e açucar,Aquisição de café e açúcar para suprir necessi...,2025,2025-01-02T00:00:00,2025-12-31T00:00:00,363,2,NaN,2024-06-27T08:13:15.608557,2024-06-27T08:13

In [6]:
import os, re
import pandas as pd

# === 1) Cópia e separação em Data/Hora ===
df_final = df.copy()
col = "dataPrevistaFormalizacaoDemanda"

if col in df_final.columns:
    dts = pd.to_datetime(df_final[col], errors="coerce")

    # Data como datetime64[ns] (não texto) -> Excel reconhece como data
    df_final[f"{col}_Data"] = dts.dt.normalize()      # 00:00:00 do mesmo dia

    # Hora como objeto datetime.time -> Excel reconhece como hora
    df_final[f"{col}_Hora"] = dts.dt.time

    # Reposiciona as novas colunas logo após a original
    cols = list(df_final.columns)
    i = cols.index(col)
    for newc in [f"{col}_Data", f"{col}_Hora"]:
        cols.remove(newc)
        i += 1
        cols.insert(i, newc)
    df_final = df_final.loc[:, cols]
else:
    print(f'Coluna "{col}" não encontrada.')

# Preview (opcional)
from IPython.display import display
print(f"Linhas: {len(df_final)} | Colunas: {df_final.shape[1]}")
print(df_final[[col, f"{col}_Data", f"{col}_Hora"]].dtypes)
display(df_final.head(20))

# === 2) Limpeza leve para Excel (sem mexer na Hora) ===
illegal = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")
for c in df_final.select_dtypes(include=["object"]).columns:
    if c != f"{col}_Hora":  # não converter a Hora para string
        df_final[c] = df_final[c].astype("string").str.replace(illegal, "", regex=True)

from openpyxl.utils import get_column_letter

PASTA = r"C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\5 - PGC"
os.makedirs(PASTA, exist_ok=True)
ARQUIVO = os.path.join(PASTA, "pgc_detalhe_catalogo_ajustado.xlsx")

with pd.ExcelWriter(ARQUIVO, engine="openpyxl") as writer:
    df_final.to_excel(writer, index=False, sheet_name="dados")

    ws = writer.sheets["dados"]

    # localiza índices (1-based no Excel)
    col_idx_date = df_final.columns.get_loc(f"{col}_Data") + 1
    col_idx_time = df_final.columns.get_loc(f"{col}_Hora") + 1

    # Larguras
    ws.column_dimensions[get_column_letter(col_idx_date)].width = 12
    ws.column_dimensions[get_column_letter(col_idx_time)].width = 10

    # Formatação numérica (data/hora) em todas as linhas (pula o header)
    for row in range(2, len(df_final) + 2):
        ws.cell(row=row, column=col_idx_date).number_format = "DD/MM/YYYY"
        ws.cell(row=row, column=col_idx_time).number_format = "HH:MM:SS"

print(f"📊 XLSX salvo em: {ARQUIVO}")

    


Linhas: 2501 | Colunas: 57
dataPrevistaFormalizacaoDemanda         string[python]
dataPrevistaFormalizacaoDemanda_Data    datetime64[ns]
dataPrevistaFormalizacaoDemanda_Hora            object
dtype: object


,codigoUasg,nomeUasg,orgao,numeroArtefato,anoArtefato,codigoEstadoArtefato,codigoCategoriaArtefato,descricaoArtefato,codigoTipoArtefato,ordemDfd,descricaoObjetoDfd,nivelPrioridadeDfd,dataPrevistaFormalizacaoDemanda,dataPrevistaFormalizacaoDemanda_Data,dataPrevistaFormalizacaoDemanda_Hora,codigoAreaDfd,tipoItem,itemSustentavel,codigoGrupoMaterial,nomeGrupoMaterial,codigoClasseMaterial,nomeClasseMaterial,codigoPdmMaterial,nomePdmMaterial,codigoSecaoServico,nomeSecaoServico,codigoDivisaoServico,nomeDivisaoServico,codigoGrupoServico,nomeGrupoServico,codigoClasseServico,nomeClasseServico,codigoSubclasseServico,nomeSubclasseServico,codigoItemCatalogo,descricaoItemCatalogo,siglaUnidadeFornecimento,nomeUnidadeFornecimento,quantidadeItem,valorUnitarioItem,valorTotalItem,tituloProjetoCompra,descricaoProjetoCompra,anoPcaProjetoCompra,dataInicioProcessoCompra,dataFimProcessoCompra,duracaoProcessoCompra,numeroItemPncp,statusContratacaoExecucao,dataHoraPublicacaoPncp,dataHoraAtualizacaoArtefato,dataHoraAtualizacaoProjetoCompra,dataHoraAtualizacaoDfd,dataHoraAtualizacaoItem,_ano,_tipo,_codigo
0,928257,CONSELHO DE ARQUITETURA E URBANISMO - PR,14804099000199,1,2025,9,1,1/2025,5,1,Aquisição de matérias de consumo diversos e ut...,1.0,2025-07-30T00:00:00,2025-07-30,00:00:00,44523,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,618302,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , TIPO: ÚNIC...",CX,Caixa,300.0,35.0000,10500.000,Aquisição material alimenticio - Ca´fé,"Aquisição de café moído torrado gourmet, embal...",2025,2025-02-03T00:00:00,2025-03-31T00:00:00,56,1,NaN,2025-01-28T10:07:10.702327,2025-01-28T10:07:10.771,2025-06-06T15:24:38,2025-01-28T10:07:09,2025-01-28T10:07:08.97774,2025,Material,8955
1,156678,UNIVERSIDADE FEDERAL DE JATAI,35840659000130,6,2025,9,1,6/2025,5,2,SeTI-Materiais de Copa,1.0,2025-07-31T00:00:00,2025-07-31,00:00:00,14348,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463571,"CAFÉ, APRESENTAÇÃO: TORRADO EM GRÃO , INTENSID...",PCT,Pacote,25.0,50.0000,1250.000,Material de Expediente Seti,"Louças, materiais de higiene",2025,2025-02-20T00:00:00,2025-05-28T00:00:00,97,1,NaN,2024-06-13T15:53:56.061181,2024-06-13T15:53:56.145,2024-11-27T15:43:12,2025-05-06T17:32:31,2024-06-13T15:53:54.230632,2025,Material,8955
2,929966,CAMARA MUNICIPAL DE CORDEIRO - RJ,42498600000171,35,2025,9,1,35/2025,5,1,"Aquisição de Material de Copa, Cozinha, Gênero...",0.0,2025-05-30T00:00:00,2025-05-30,00:00:00,46426,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463591,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , INTENSIDAD...",PCT,Pacote,60.0,21.0000,1260.000,Aquisição de Material de Copa e Cozinha,<NA>,2025,2025-03-20T00:00:00,2025-06-04T00:00:00,76,1,NaN,2024-05-16T13:45:30.090226,2024-05-16T13:45:30.601,2025-01-10T14:55:51,2024-05-16T13:45:29,2024-05-16T13:45:29.154059,2025,Material,8955
3,380197,ESP-PENITENCIARIA I DE SERRA AZUL,96291141000180,6,2025,9,1,6/2025,5,2,AQUISIÇÃO DE GÊNEROS ALIMENTÍCIOS ESTOCÁVEIS P...,2.0,2025-04-30T00:00:00,2025-04-30,00:00:00,39834,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19765,ACHOCOLATADO,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463556,"ACHOCOLATADO, APRESENTAÇÃO: PÓ , SABOR: TRADIC...",EMB,Embalagem,72.0,15.4680,1113.696,GENEROS ALIMENTÍCIOS - JAN-ABR-25,<NA>,2025,2024-10-25T00:00:00,2025-01-02T00:00:00,69,2,NaN,2024-06-11T15:28:13.12969,2024-06-11T15:28:13.566,2024-06-18T16:11:36,2024-12-12T10:52:05,2024-12-12T10:51:46.323878,2025,Material,8955
4,193001,DNOCS 1A.DR - TERESINA-PI,00043711000143,18,2025,9,1,18/2025,5,1,Aquisição de café e açúcar para a CEST/PI.,0.0,2025-12-31T00:00:00,2025-12-31,00:00:00,7704,M,False,89,SUBSISTÊNCIA,8955,"CAFÉ, CHÁ E CHOCOLATE",19766,CAFÉ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,463587,"CAFÉ, APRESENTAÇÃO: TORRADO MOÍDO , INTENSIDAD...",PCT,Pacote,200.0,11.0000,2200.000,A

📊 XLSX salvo em: C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\5 - PGC\pgc_detalhe_catalogo_ajustado.xlsx
